# OpenAI Streaming - Traced With LastMile

Four use cases:

1. Text - Non-streaming
2. [this one] Text - Streaming
3. Tool Calls - Non-streaming
4. Tool Calls - Streaming

## How to generate function arguments

In [1]:
# !pip install scipy --quiet
# !pip install tenacity --quiet
# !pip install tiktoken --quiet
# !pip install termcolor --quiet
# !pip install openai --quiet
# !pip install "tracing-auto-instrumentation[openai]"
!pip list | grep tracing_auto_instrumentation

# Create ~/.env file with this line: OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

tracing_auto_instrumentation             0.0.4     /Users/rossdancraig/Projects/tracing_auto_instrumentation


In [2]:
import openai

from lastmile_eval.rag.debugger.api.tracing import LastMileTracer

from tracing_auto_instrumentation.openai import openai as openai_tracing
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="OpenAI Text Calling w. Streaming",
)
client = openai.OpenAI(api_key=OPENAI_API_KEY)
client = openai_tracing.wrap(client, tracer)

/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-29 15:11:55,666 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:55,779 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=OpenAI+Text+Calling+w.+Streaming HTTP/1.1" 200 367
2024-05-29 15:11:55,781 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-29 15:11:55,782 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'


In [3]:
def run_my_existing_openai_app(user_message: str, stream: bool = True):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": stream,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = client.chat.completions.create(**completion_params)
    print("Chat Completion Response: ")
    if stream:
        for chunk in response:
            print(chunk)
    else:
        print(response)

In [4]:
# Run your code as usual
run_my_existing_openai_app("Tell me a joke about apples")

2024-05-29 15:11:55,809 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Tell me a joke about apples', 'role': 'user'}], 'model': 'gpt-3.5-turbo', 'max_tokens': 3000, 'stream': True, 'temperature': 1, 'top_p': 1}}
2024-05-29 15:11:55,820 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-05-29 15:11:55,821 - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
2024-05-29 15:11:55,861 - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x168e9af60>
2024-05-29 15:11:55,861 - start_tls.started ssl_context=<ssl.SSLContext object at 0x168ea7750> server_hostname='api.openai.com' timeout=5.0
2024-05-29 15:11:55,877 - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x168d34890>
2024-05-29 15:11:55,877 - send_request_headers.started request=<Request [b'POST']>
2024-05-29 15:11:55,878 - send_

Chat Completion Response: 
fuck this starting create now
starting this stupid dumbass span


2024-05-29 15:11:56,305 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 29 May 2024 19:11:56 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'147'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1996992'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'90ms'), (b'x-request-id', b'req_f26ddc0caeb01b3e1a741dacb40cfdfb'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=vKjWHAnFACZVrNAY8agIROhqlKfMaANG5O3nL6oXLVw-1717009916-1.0.1.1-i7yFPceqitnAKe88CVa12wl1SynzSilKiBg35JpUVn_v0pX0p.r7fBTVxpamShE_RHGRwP63FR2to_vUsBAhbg; path=/; expires

ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='Why', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' did', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChu

2024-05-29 15:11:56,689 - receive_response_body.complete
2024-05-29 15:11:56,690 - response_closed.started
2024-05-29 15:11:56,690 - response_closed.complete
2024-05-29 15:11:56,695 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:56,845 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-29 15:11:56,850 - Starting new HTTPS connection (1): lastmileai.dev:443


ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' doctor', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='?', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' Because', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionC

2024-05-29 15:11:56,942 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 475
2024-05-29 15:11:56,944 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:57,146 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 808
